In [1]:
from data_utils import load_data, preprocess_data_faiss, preprocess_data_faiss_test

In [2]:
data = load_data("data")

In [3]:
embeddings = preprocess_data_faiss(data)
embeddings.shape

/mnt/c/Users/Raman/PycharmProjects/stackexchange-recsys/data_utils.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts["body"] = posts["body"].map(clean_html)


(11812, 2048)

In [4]:
posts, test_data = preprocess_data_faiss_test(data)
test_data

/mnt/c/Users/Raman/PycharmProjects/stackexchange-recsys/data_utils.py:244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts["id"] = posts["id"].astype("int32")


,postid,relatedpostid,linktypeid
1,8619,2788,1
3,4808,8943,1
4,3116,3070,1
5,1215,398,1
6,3174,117,1
...,...,...,...
4266,9742,8129,1
4267,11603,8329,1
4269,1296,155,1
4270,9570,5524,1


In [5]:
import faiss
import numpy as np

In [19]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)
test_embeddings = embeddings[test_data["postid"]]

In [38]:
random_embedding = test_embeddings[
    np.random.choice(test_embeddings.shape[0])
].reshape(1, -1)

In [39]:
print("IndexFlatL2:")
for k in range(6, 22, 5):
    D, I = index.search(test_embeddings, k=k)

    correct = 0

    for related_post_id, recommendations in zip(test_data["relatedpostid"], I):
        if related_post_id in I:
            correct += 1

    print(f"Recall@k (k = {k - 1}): {correct / test_data.shape[0]}")

IndexFlatL2:
Recall@k (k = 5): 0.8853914447134786
Recall@k (k = 10): 0.9461931665321496
Recall@k (k = 15): 0.9714823782620393
Recall@k (k = 20): 0.9819747107882701


In [40]:
D, I = index.search(random_embedding, k=6)

print("IndexFlatL2:")
print(f"Distances: {D}")
print(f"Indices: {I}\n")

print(
    f"===ORIGINAL POST===\n"
    f"TITLE: {posts['title'].iloc[I[0][0]]}\n"
    f"CONTENT: {posts['body'].iloc[I[0][0]]}"
)

for i in range(1, 6):
    print(
        f"===RECOMMENDATION {i} (distance = {D[0][i]})===\n"
        f"TITLE: {posts['title'].iloc[I[0][i]]}\n"
        f"CONTENT: {posts['body'].iloc[I[0][i]]}"
    )

IndexFlatL2:
Distances: [[0.         0.34699026 0.3767523  0.38039908 0.40704072 0.43568596]]
Indices: [[9305 6938 7431 9138 6947 7951]]

===ORIGINAL POST===
TITLE: cooking oven temperature
CONTENT: I have a murphy electric cake oven. it is a classic electric cake oven(not a modern one) In the recipe book it is mentioned to preheat it to 200 degree C for 10 mins. But it is not specified anywhere what should the temperature of the oven be while baking. My cake often gets overcooked/burnt if i keep a high temperature. or remains uncooked if i keep it low. Can anyone specify a temp. for baking a cake or a brownie please.

===RECOMMENDATION 1 (distance = 0.3469902575016022)===
TITLE: How hot is your oven?
CONTENT: 
There's also an answer here:
Can I test my oven temperature without an oven thermometer?

I have a big dispute with an electrician if the oven he supposedly fixed works or not. Problem is I think it reaches only about 150 C (300F) degrees when it indicates it has reached 250 C. 

In [41]:
from sklearn.decomposition import PCA

In [42]:
pca = PCA(n_components=128)
reduced_embeddings = pca.fit_transform(embeddings)

In [43]:
random_embedding = reduced_embeddings[
    I[0][0]
].reshape(1, -1)

In [44]:
index = faiss.IndexFlatL2(reduced_embeddings.shape[1])
index.add(reduced_embeddings)
test_embeddings = reduced_embeddings[test_data["postid"]]

print("IndexFlatL2 (PCA):")
for k in range(6, 22, 5):
    D, I = index.search(test_embeddings, k=k)

    correct = 0

    for related_post_id, recommendations in zip(test_data["relatedpostid"], I):
        if related_post_id in I:
            correct += 1

    print(f"Recall@k (k = {k - 1}): {correct / test_data.shape[0]}")

IndexFlatL2 (PCA):
Recall@k (k = 5): 0.8843153080441216
Recall@k (k = 10): 0.9486144740382029
Recall@k (k = 15): 0.9706752757600215
Recall@k (k = 20): 0.9822437449556094


In [45]:
D, I = index.search(random_embedding, k=6)

print("IndexFlatL2 (PCA):")
print(f"Distances: {D}")
print(f"Indices: {I}\n")

print(
    f"===ORIGINAL POST===\n"
    f"TITLE: {posts['title'].iloc[I[0][0]]}\n"
    f"CONTENT: {posts['body'].iloc[I[0][0]]}"
)

for i in range(1, 6):
    print(
        f"===RECOMMENDATION {i} (distance = {D[0][i]})===\n"
        f"TITLE: {posts['title'].iloc[I[0][i]]}\n"
        f"CONTENT: {posts['body'].iloc[I[0][i]]}"
    )

IndexFlatL2 (PCA):
Distances: [[0.         0.35184336 0.37463248 0.3952999  0.41516256 0.4362808 ]]
Indices: [[9305 6938 7431 9138 6947 7951]]

===ORIGINAL POST===
TITLE: cooking oven temperature
CONTENT: I have a murphy electric cake oven. it is a classic electric cake oven(not a modern one) In the recipe book it is mentioned to preheat it to 200 degree C for 10 mins. But it is not specified anywhere what should the temperature of the oven be while baking. My cake often gets overcooked/burnt if i keep a high temperature. or remains uncooked if i keep it low. Can anyone specify a temp. for baking a cake or a brownie please.

===RECOMMENDATION 1 (distance = 0.35184335708618164)===
TITLE: How hot is your oven?
CONTENT: 
There's also an answer here:
Can I test my oven temperature without an oven thermometer?

I have a big dispute with an electrician if the oven he supposedly fixed works or not. Problem is I think it reaches only about 150 C (300F) degrees when it indicates it has reached 

In [46]:
index = faiss.IndexLSH(reduced_embeddings.shape[1], 64)
index.add(reduced_embeddings)
test_embeddings = reduced_embeddings[test_data["postid"]]

print("IndexLSH (PCA, n_bits=64):")
for k in range(6, 22, 5):
    D, I = index.search(test_embeddings, k=k)

    correct = 0

    for related_post_id, recommendations in zip(test_data["relatedpostid"], I):
        if related_post_id in I:
            correct += 1

    print(f"Recall@k (k = {k - 1}): {correct / test_data.shape[0]}")

IndexLSH (PCA, n_bits=64):
Recall@k (k = 5): 0.8501479687920366
Recall@k (k = 10): 0.9440408931934355
Recall@k (k = 15): 0.9712133440947001
Recall@k (k = 20): 0.9835889157923057


In [47]:
D, I = index.search(random_embedding, k=6)

print("IndexLSH (PCA, n_bits=64):")
print(f"Distances: {D}")
print(f"Indices: {I}\n")

print(
    f"===ORIGINAL POST===\n"
    f"TITLE: {posts['title'].iloc[I[0][0]]}\n"
    f"CONTENT: {posts['body'].iloc[I[0][0]]}"
)

for i in range(1, 6):
    print(
        f"===RECOMMENDATION {i} (distance = {D[0][i]})===\n"
        f"TITLE: {posts['title'].iloc[I[0][i]]}\n"
        f"CONTENT: {posts['body'].iloc[I[0][i]]}"
    )

IndexLSH (PCA, n_bits=64):
Distances: [[ 0. 13. 14. 14. 14. 15.]]
Indices: [[9305 7431 1637 2187 2453 1167]]

===ORIGINAL POST===
TITLE: cooking oven temperature
CONTENT: I have a murphy electric cake oven. it is a classic electric cake oven(not a modern one) In the recipe book it is mentioned to preheat it to 200 degree C for 10 mins. But it is not specified anywhere what should the temperature of the oven be while baking. My cake often gets overcooked/burnt if i keep a high temperature. or remains uncooked if i keep it low. Can anyone specify a temp. for baking a cake or a brownie please.

===RECOMMENDATION 1 (distance = 13.0)===
TITLE: Right baking temperature
CONTENT: Problem with the right baking temperature. I have a gas oven. To bake a cake I pre heat the oven at 150 C for at least 1/2 an hour while I prepare all the stuff required for preparing the cake . After baking the cake for 45 minutes and checking the cake with the tooth pick which comes out clean. I put off the gas and 